In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch 
from src import AllModule

state_dict = torch.load('checkpoints/all-val_error=0.70873-epoch=3.ckpt')['state_dict']
model = AllModule(dict(backbone='resnet18', pretrained=True, mlp_layers=[256, 512], mlp_dropout=0.))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [11]:
from tqdm import tqdm
from src import AllDataModule

model.cuda()

tta_trans = [
    None,
    {'HorizontalFlip': {'p': 1}},
    {'VerticalFlip': {'p': 1}},
    {'Transpose': {'p': 1}},
    {'RandomRotate90': {'p': 1}},
]

dm = AllDataModule(batch_size=512, num_workers=10, pin_memory=True)
dm.setup()

all_probas = []
for r, trans in enumerate(tta_trans):
    dm.test_trans = trans 
    dm.generate_datasets()
    probas, observations = torch.tensor([]), []
    for batch in tqdm(dm.test_dataloader()):
        preds = model.predict(batch)
        probas = torch.concat([probas, preds.cpu()], dim=0)
        observation_ids = batch['observation_id']
        observations += observation_ids.cpu().numpy().tolist()
    all_probas.append(probas)
all_probas = torch.stack(all_probas, dim=0) # 5, N, D

train: 1587395
val: 40080
test: 36421


100%|██████████| 72/72 [00:49<00:00,  1.44it/s]


Compose([
  HorizontalFlip(always_apply=False, p=1),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={'nir': 'image', 'alt': 'image', 'lc': 'image'})


 15%|█▌        | 11/72 [00:11<01:04,  1.05s/it]


KeyboardInterrupt: 

In [8]:
mean_probas = all_probas.mean(dim=0) # N, D
values, ixs = mean_probas.topk(30)
final_labels = [' '.join([str(i.item()) for i in ix]) for ix in ixs]

In [9]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': final_labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
5827,10683833,912 155 554 386 736 169 715 440 949 813 298 12...
10138,10189873,155 912 152 5 1074 240 754 581 108 106 677 440...
22771,20594215,224 2076 5095 5045 523 129 2950 2225 5612 5503...
25136,20858303,5053 5095 131 389 720 5045 1042 1015 5228 1632...
36356,22063459,8290 5143 5130 4935 933 3495 7395 5205 5 1458 ...


In [10]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
11777,10563762,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
10089,10482124,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
14068,10671561,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
9823,10467756,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
20394,20341750,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1504,10072323,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
13993,10668344,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
19505,20246222,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
31226,21511549,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
12334,10592213,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
